# 1차 분류기 모델(가정)

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib  # 모델 저장용

## 데이터 로딩

In [4]:
df = pd.read_csv("./dataset/must_use_final.csv")

X = df[["LAT", "LON", "COG", "HEADING"]]
y = df["CLUSTER_1"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

## 모델별 파라미터 설정

In [14]:
knn_params = {
    "n_neighbors": 7,            # 더 많은 이웃 → 일반화 잘됨
    "weights": "distance",       # 가까운 이웃에 더 큰 영향력
    "p": 2                       # 유클리디안 거리 (기본값)
}
xgb_params = {
    "use_label_encoder": False,
    "eval_metric": "mlogloss",
    "n_estimators": 200,
    "max_depth": 6,
    "learning_rate": 0.05,
    "subsample": 0.8,            # 일부 샘플 사용 (과적합 방지)
    "colsample_bytree": 0.8,     # 일부 피처 사용 (과적합 방지)
    "reg_lambda": 5,             # L2 정규화로 모델 안정화
    "gamma": 1,                  # 분할 조건 강화 → 과적합 방지
    "random_state": 42
}

rf_params = {
    "n_estimators": 300,         # 더 많은 트리로 안정적인 예측
    "max_depth": 15,             # 깊이를 제한해 과적합 방지
    "min_samples_split": 5,      # 분할 최소 샘플 수 조절
    "min_samples_leaf": 2,       # 리프 노드 최소 샘플 수
    "max_features": "sqrt",      # 트리 다양성 확보
    "class_weight": "balanced",  # 불균형 클래스 대응
    "random_state": 42
}

## 모델 정의

In [16]:
model_rf = RandomForestClassifier(**rf_params)
model_knn = KNeighborsClassifier(**knn_params)
model_xgb = XGBClassifier(**xgb_params)

In [18]:
voting_model = VotingClassifier(
    estimators=[
        ("rf", model_rf),
        ("knn", model_knn),
        ("xgb", model_xgb)
    ],
    voting="soft"
)

## 모델 학습

In [20]:
voting_model.fit(X_train, y_train)

C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:39:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=15,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=300,
                                                     random_state=42)),
                             ('knn',
                              KNeighborsClassifier(n_neighbors=7,
                                                   weights='distance')),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsampl...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [22]:
y_pred = voting_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")

In [23]:
print("🎯 1차 분류기 평가 결과:")
print(f"- Accuracy: {acc:.4f}")
print(f"- Macro F1-score: {f1:.4f}")

# 9. 교차검증 평가
cv_scores = cross_val_score(voting_model, X, y, cv=5, scoring="f1_macro")
print(f"\n📈 5-Fold CV Macro F1 평균: {cv_scores.mean():.4f}")

# 10. 모델 저장
joblib.dump(voting_model, "softvoting_cluster1_model.joblib")
print("\n✅ 모델이 softvoting_cluster1_model.joblib 으로 저장되었습니다.")

🎯 1차 분류기 평가 결과:
- Accuracy: 0.9034
- Macro F1-score: 0.8880


C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:42:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:45:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:48:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgbo


📈 5-Fold CV Macro F1 평균: 0.7865

✅ 모델이 softvoting_cluster1_model.joblib 으로 저장되었습니다.


# 2차 분류기

In [6]:
import pandas as pd

# 1. 데이터 불러오기
df = pd.read_csv("./dataset/cluster1_2.csv")

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df = df.sort_values(by=['VSL_ID', 'PORT_NAME', 'TIMESTAMP']).reset_index(drop=True)

# 결과 저장 리스트
rows_5h = []

# 30번째 시점 추출 (10분 단위 × 30 = 5시간)
for (vsl_id, port), group in df.groupby(['VSL_ID', 'PORT_NAME']):
    if len(group) >= 30:
        row_5h = group.iloc[29]  # 인덱스는 0부터 시작이므로 29번째가 30번째 행
        rows_5h.append(row_5h)

# 최종 데이터프레임
df_5h = pd.DataFrame(rows_5h).reset_index(drop=True)
print(df_5h.shape)
display(df_5h.head())
display(df_5h['PORT_NAME'].value_counts())

(91, 11)


COUNTRY PORT_NAME                                VSL_ID           TIMESTAMP  \
0      CN     CNNGB  060db0be-c97f-3f11-8304-c637fe4fa4d5 2024-10-01 15:40:00   
1      CN     CNSHA  06724e0f-5a08-3aa8-b42c-97acf4f8102d 2024-10-26 23:40:00   
2      CN     CNSHA  06e20c85-1dab-34e7-bbdb-6ed30bd68672 2024-11-09 16:20:00   
3      CN     CNSHA  0986a961-7abf-3ade-9b02-40c5323168aa 2024-07-21 14:00:00   
4      CN     CNSHA  0f09bfcc-ce6f-3096-a547-5618e8e76d29 2024-11-03 19:00:00   

      COG  HEADING        LAT         LON PORT_CD  CLUSTER_1  CLUSTER_2  
0  239.30    240.0  34.515247  128.546545   CNNGB          2          0  
1  227.50    230.0  34.058808  128.528450   CNSHA          2          2  
2  202.80    205.0  34.239492  128.754670   CNSHA          2          2  
3  224.30    223.0  34.363648  128.308899   CNSHA          2          2  
4  239.15    238.5  34.251924  128.100558   CNSHA          2          2

PORT_NAME
CNSHA    72
CNNGB    11
CNTAC     4
CNNJI     4
Name: count, dtype: int64

In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import joblib
import os

In [74]:
df = pd.read_csv("./dataset/cluster1_2.csv")
X = df[["LAT", "LON", "COG", "HEADING"]]
le = LabelEncoder()
y = le.fit_transform(df["PORT_NAME"])

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [78]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [4, 6],
    "learning_rate": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

In [80]:
grid = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42),
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:57:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.05, 0.1], 'max_depth': [4, 6],
                         'n_estimators': [100, 200], 'subsample': [0.8, 1.0]},
             scoring='f1_macro')

In [81]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="macro")

In [82]:
print("✅ 군집 2 XGBoost 모델 평가 결과:")
print(f" - Accuracy: {acc:.4f}")
print(f" - Macro F1: {f1:.4f}")
print(" - Best Params:", grid.best_params_)

# 6. 모델과 LabelEncoder 저장
joblib.dump(best_model, "models/port_model_7_xgb.joblib")
joblib.dump(le, "models/encoder_7.joblib")
print("\n💾 XGBoost 모델과 레이블 인코더가 저장되었습니다.")

✅ 군집 2 XGBoost 모델 평가 결과:
 - Accuracy: 0.9365
 - Macro F1: 0.7532
 - Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200, 'subsample': 0.8}

💾 XGBoost 모델과 레이블 인코더가 저장되었습니다.


# 오버 샘플링 - 2번 군집

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from collections import defaultdict
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
import joblib
import numpy as np
import os

# 2. Feature / Target 설정
X = df_5h[["LAT", "LON", "COG", "HEADING"]]
le = LabelEncoder()
y = le.fit_transform(df_5h["PORT_NAME"])

# ✅ 3. 오버샘플링 수행
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# 4. Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, stratify=y_resampled, test_size=0.2, random_state=42
)
# 저장 폴더가 없으면 생성
os.makedirs("datasets", exist_ok=True)

# 저장 실행
np.save("datasets/X_train_cluster2.npy", X_train)
np.save("datasets/X_test_cluster2.npy", X_test)
np.save("datasets/y_train_cluster2.npy", y_train)
np.save("datasets/y_test_cluster2.npy", y_test)

# 5. XGBoost 하이퍼파라미터 튜닝
param_grid = {
    "n_estimators": [50, 100],          # ✅ 트리 수 줄이기
    "max_depth": [2, 3, 4],             # ✅ 트리 깊이 제한
    "learning_rate": [0.01, 0.05],      # ✅ 천천히 학습
    "subsample": [0.6, 0.8],            # ✅ 전체 데이터 일부만 사용
    "colsample_bytree": [0.6, 0.8],     # ✅ 피처 일부만 사용
    "gamma": [1, 3],                    # ✅ 분할 최소 gain 증가
    "reg_lambda": [1, 5],               # ✅ L2 정규화 (규제 강화)
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric="mlogloss",
    random_state=42
)

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,            # 👉 여기 들어감!
    n_jobs=-1
)

grid.fit(X_train, y_train)
best_model = grid.best_estimator_

# 7. 테스트 평가
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average="macro")

print("\n✅ [TEST SET] 성능")
print(f" - Accuracy: {acc:.4f}")
print(f" - Macro F1: {macro_f1:.4f}")
print(" - Best Params:", grid.best_params_)

# 8. 교차검증
print("\n🔁 [CV] 교차검증 수행 중...")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_cv = cross_val_predict(best_model, X_resampled, y_resampled, cv=skf)

acc_cv = accuracy_score(y_resampled, y_pred_cv)
macro_f1_cv = f1_score(y_resampled, y_pred_cv, average='macro')
weighted_f1_cv = f1_score(y_resampled, y_pred_cv, average='weighted')

print(f"\n✅ [CV] Accuracy: {acc_cv:.4f}")
print(f"🌍 [CV] Macro F1-score: {macro_f1_cv:.4f}")
print(f"⚖️ [CV] Weighted F1-score: {weighted_f1_cv:.4f}")

# 9. 클래스별 정확도 출력
print("\n📌 [CV] CLUSTER_2 항구별 정확도:")
class_accuracy_cv = defaultdict(list)
for true_label, pred_label in zip(y_resampled, y_pred_cv):
    class_accuracy_cv[true_label].append(true_label == pred_label)

for cls in sorted(class_accuracy_cv):
    acc = np.mean(class_accuracy_cv[cls])
    port_name = le.inverse_transform([cls])[0]
    print(f" - {port_name}: 정확도 {acc:.4f} ({len(class_accuracy_cv[cls])}개 샘플)")

# 10. classification report
print("\n📊 [CV] Classification Report:")
print(classification_report(y_resampled, y_pred_cv, target_names=le.inverse_transform(np.unique(y_resampled)), digits=4))

# 11. 과적합 분석
print("\n🔍 [과적합 분석]")
y_pred_train = best_model.predict(X_train)
train_acc = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average="macro")

print(f" - Train Accuracy : {train_acc:.4f}")
print(f" - Test Accuracy  : {acc:.4f}")
print(f" - Train Macro F1 : {train_f1:.4f}")
print(f" - Test Macro F1  : {macro_f1:.4f}")

acc_gap = train_acc - acc
f1_gap = train_f1 - macro_f1
print(f"\n⚠️ [과적합 지표]")
print(f" - Accuracy 차이 (Train - Test): {acc_gap:.4f}")
print(f" - Macro F1 차이 (Train - Test): {f1_gap:.4f}")
print("✅ 과적합 문제 없음!" if acc_gap <= 0.1 and f1_gap <= 0.1 else "❗ 과적합 가능성 있음!")

C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:38:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ [TEST SET] 성능
 - Accuracy: 0.8448
 - Macro F1: 0.8236
 - Best Params: {'colsample_bytree': 0.6, 'gamma': 1, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.8}

🔁 [CV] 교차검증 수행 중...


C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:38:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:38:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:38:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\xgbo


✅ [CV] Accuracy: 0.9028
🌍 [CV] Macro F1-score: 0.8986
⚖️ [CV] Weighted F1-score: 0.8986

📌 [CV] CLUSTER_2 항구별 정확도:
 - CNNGB: 정확도 0.9167 (72개 샘플)
 - CNNJI: 정확도 1.0000 (72개 샘플)
 - CNSHA: 정확도 0.6944 (72개 샘플)
 - CNTAC: 정확도 1.0000 (72개 샘플)

📊 [CV] Classification Report:
              precision    recall  f1-score   support

       CNNGB     0.8462    0.9167    0.8800        72
       CNNJI     0.9474    1.0000    0.9730        72
       CNSHA     0.8929    0.6944    0.7812        72
       CNTAC     0.9231    1.0000    0.9600        72

    accuracy                         0.9028       288
   macro avg     0.9024    0.9028    0.8986       288
weighted avg     0.9024    0.9028    0.8986       288


🔍 [과적합 분석]
 - Train Accuracy : 0.9739
 - Test Accuracy  : 1.0000
 - Train Macro F1 : 0.9734
 - Test Macro F1  : 0.8236

⚠️ [과적합 지표]
 - Accuracy 차이 (Train - Test): -0.0261
 - Macro F1 차이 (Train - Test): 0.1498
❗ 과적합 가능성 있음!


# 찐 최종

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from collections import defaultdict

# 1. 데이터 로딩
df = pd.read_csv("./dataset/cluster1_2.csv")
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"])
df = df.sort_values(["VSL_ID", "PORT_NAME", "TIMESTAMP"])

# 2. 5시간 시점 추출
rows_5h = []
for (vsl_id, port), group in df.groupby(["VSL_ID", "PORT_NAME"]):
    if len(group) >= 30:
        rows_5h.append(group.iloc[29])
df_5h = pd.DataFrame(rows_5h).reset_index(drop=True)

# 3. 특성과 타겟
X = df_5h[["LAT", "LON", "COG", "HEADING"]]
le = LabelEncoder()
y = le.fit_transform(df_5h["PORT_NAME"])

# 4. Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# 5. GridSearch 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# XGBoost
xgb_grid = GridSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42),
    param_grid={
        "n_estimators": [50, 100],
        "max_depth": [3, 5],
        "learning_rate": [0.05, 0.1]
    },
    cv=cv, scoring="f1_macro", n_jobs=-1
)
xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

# RandomForest (with class_weight)
rf_grid = GridSearchCV(
    RandomForestClassifier(class_weight="balanced", random_state=42),
    param_grid={
        "n_estimators": [50, 100],
        "max_depth": [3, 5]
    },
    cv=cv, scoring="f1_macro", n_jobs=-1
)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

# KNN
knn_grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid={
        "n_neighbors": [3, 5],
        "weights": ["uniform", "distance"],
        "p": [1, 2]
    },
    cv=cv, scoring="f1_macro", n_jobs=-1
)
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_

# 6. Soft Voting 구성 및 학습
voting_clf = VotingClassifier(
    estimators=[("xgb", best_xgb), ("rf", best_rf), ("knn", best_knn)],
    voting="soft"
)
voting_clf.fit(X_train, y_train)

# 7. 평가
y_pred = voting_clf.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred, average="macro")
print(f"✅ [Test Set] Accuracy: {test_acc:.4f}, Macro F1: {test_f1:.4f}")

# 8. 교차검증
y_pred_cv = cross_val_predict(voting_clf, X, y, cv=cv)
cv_acc = accuracy_score(y, y_pred_cv)
cv_macro_f1 = f1_score(y, y_pred_cv, average="macro")
print(f"📊 [CV] Accuracy: {cv_acc:.4f}, Macro F1: {cv_macro_f1:.4f}")

# 9. 클래스별 정확도 출력
class_accuracy = defaultdict(list)
for true, pred in zip(y, y_pred_cv):
    class_accuracy[true].append(true == pred)

print("\n📌 [클래스별 정확도]")
for cls in sorted(class_accuracy):
    port_name = le.inverse_transform([cls])[0]
    acc = np.mean(class_accuracy[cls])
    print(f" - {port_name}: 정확도 {acc:.4f} ({len(class_accuracy[cls])}개 샘플)")


✅ [Test Set] Accuracy: 0.7368, Macro F1: 0.2121
📊 [CV] Accuracy: 0.7253, Macro F1: 0.2102

📌 [클래스별 정확도]
 - CNNGB: 정확도 0.0000 (11개 샘플)
 - CNNJI: 정확도 0.0000 (4개 샘플)
 - CNSHA: 정확도 0.9167 (72개 샘플)
 - CNTAC: 정확도 0.0000 (4개 샘플)


In [ ]:
import joblib
import os

# 모델 저장
joblib.dump(best_model, "models/port.joblib")

# 라벨 인코더 저장
joblib.dump(le, "models/cluster_2_label_encoder.joblib")

print("✅ 모델과 레이블 인코더가 저장되었습니다!")


In [ ]:
import joblib

# 모델 저장
joblib.dump(best_model, "models/port_model_2_os.joblib")

# LabelEncoder 저장 (같이 써야 해!)
joblib.dump(le, "models/encoder_2_os.joblib")

print("✅ 오버샘플링된 모델과 라벨 인코더가 저장되었습니다!")

# 가정

In [85]:
import numpy as np
import joblib

def predict_next_port_interactive():
    print("🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)")
    try:
        lat = float(input("LAT (위도): "))
        lon = float(input("LON (경도): "))
        cog = float(input("COG (침로): "))
        heading = float(input("HEADING (타각): "))
    except ValueError:
        print("⚠️ 숫자만 입력해 주세요.")
        return

    user_input = np.array([[lat, lon, cog, heading]])

    # 1차 분류기 로드
    try:
        cluster_model = joblib.load("models/softvoting_cluster1_model.joblib")
    except FileNotFoundError:
        print("❌ 1차 분류기 모델 파일이 없습니다.")
        return

    predicted_cluster = cluster_model.predict(user_input)[0]
    print(f"\n🔍 예측된 군집(CLUSTER_1): {predicted_cluster}")

    # 2차 분류기 분기 처리 (현재는 군집 2만 구현됨)
    if predicted_cluster == 2:
        try:
            port_model = joblib.load("models/cluster_2_rf.joblib")
            le = joblib.load("models/cluster_2_label_encoder.joblib")
        except FileNotFoundError:
            print("❌ 군집 2의 2차 분류기 또는 라벨 인코더 파일이 없습니다.")
            return

        port_probs = port_model.predict_proba(user_input)[0]
        top3_idx = port_probs.argsort()[::-1][:3]
        top3_ports = le.inverse_transform(top3_idx)
        top3_probs = port_probs[top3_idx]

        print("\n🎯 군집 2 → Top-3 도착 항구 예측 결과:")
        for port, prob in zip(top3_ports, top3_probs):
            print(f"📦 {port}: {prob:.2%}")

    else:
        print("⚠️ 현재는 군집 2만 지원 중입니다. 다른 군집은 추후 구현 예정입니다.")

In [89]:
predict_next_port_interactive()

🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)


LAT (위도):  40
LON (경도):  130
COG (침로):  239.15
HEADING (타각):  235.5



🔍 예측된 군집(CLUSTER_1): 7
⚠️ 현재는 군집 2만 지원 중입니다. 다른 군집은 추후 구현 예정입니다.


C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


# 가상 모델링

In [91]:
import numpy as np
import joblib

# 🔁 군집별 모델 경로 딕셔너리
cluster_model_paths = {
    1: {"model": "models/cluster_1_rf_xgb_knn_lightgbm.joblib", "encoder": "models/cluster_1_label_encoder.joblib"},
    2: {"model": "models/cluster_2_rf.joblib", "encoder": "models/cluster_2_label_encoder.joblib"},
    7: {"model": "models/cluster_7_rf.joblib", "encoder": "models/cluster_7_label_encoder.joblib"},
    # 필요시 계속 추가 가능
}

def predict_next_port_interactive():
    print("🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)")
    try:
        lat = float(input("LAT (위도): "))
        lon = float(input("LON (경도): "))
        cog = float(input("COG (침로): "))
        heading = float(input("HEADING (타각): "))
    except ValueError:
        print("⚠️ 숫자만 입력해 주세요.")
        return

    user_input = np.array([[lat, lon, cog, heading]])

    # ▶ 1차 분류기 로드 및 CLUSTER_1 예측
    try:
        cluster_model = joblib.load("models/softvoting_cluster1_model.joblib")
    except FileNotFoundError:
        print("❌ 1차 분류기 모델이 없습니다.")
        return

    cluster = cluster_model.predict(user_input)[0]
    print(f"\n🔍 예측된 군집(CLUSTER_1): {cluster}")

    # ▶ 2차 분류기 존재 여부 확인
    if cluster not in cluster_model_paths:
        print(f"⚠️ 군집 {cluster}에 대한 2차 분류기 모델이 아직 준비되지 않았습니다.")
        return

    try:
        model = joblib.load(cluster_model_paths[cluster]["model"])
        le = joblib.load(cluster_model_paths[cluster]["encoder"])
    except FileNotFoundError:
        print("❌ 2차 분류기 또는 라벨 인코더 파일이 존재하지 않습니다.")
        return

    # ▶ 항구 예측
    probs = model.predict_proba(user_input)[0]
    top3_idx = probs.argsort()[::-1][:3]
    top3_ports = le.inverse_transform(top3_idx)
    top3_probs = probs[top3_idx]

    # ▶ 결과 출력
    print("\n Top-3 항구 예측 결과:")
    for port, prob in zip(top3_ports, top3_probs):
        print(f"  - {port}: {prob:.2%}")


In [93]:
import warnings
warnings.filterwarnings('ignore')

In [97]:
predict_next_port_interactive()

🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)


LAT (위도):  34.65685204
LON (경도):  129.6224063
COG (침로):  155.97964
HEADING (타각):  153.1197



🔍 예측된 군집(CLUSTER_1): 1

 Top-3 항구 예측 결과:
  - JPHKT: 94.25%
  - JPHIJ: 3.45%
  - KRKAN: 0.88%


# 확률 곱 전체 로직

In [77]:
import numpy as np
import joblib

# 🔁 군집별 모델 경로 딕셔너리
cluster_model_paths = {
    0: {"model": "models/cluster_0_rf.joblib", "encoder": "models/cluster_0_label_encoder.joblib"},
    1: {"model": "models/cluster_1_rf_xgb_knn_lightgbm.joblib", "encoder": "models/cluster_1_label_encoder.joblib"},
    2: {"model": "models/cluster_2_rf.joblib", "encoder": "models/cluster_2_label_encoder.joblib"},
    7: {"model": "models/cluster_7_rf.joblib", "encoder": "models/cluster_7_label_encoder.joblib"}
}

def predict_next_port_joint_prob():
    print("🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)")
    try:
        lat = float(input("LAT (위도): "))
        lon = float(input("LON (경도): "))
        cog = float(input("COG (침로): "))
        heading = float(input("HEADING (타각): "))
    except ValueError:
        print("⚠️ 숫자만 입력해 주세요.")
        return

    user_input = np.array([[lat, lon, cog, heading]])

    # ▶ 1차 분류기 로드 및 CLUSTER_1 확률 예측
    try:
        cluster_model = joblib.load("models/softvoting_cluster1_model.joblib")
    except FileNotFoundError:
        print("❌ 1차 분류기 모델이 없습니다.")
        return

    cluster_probs = cluster_model.predict_proba(user_input)[0]
    print("\n🔍 [1차 분류기] 군집별 확률:")
    for i, p in enumerate(cluster_probs):
        print(f" - CLUSTER_{i}: {p:.2%}")

    final_probs = {}

    # ▶ 각 군집별로 항구 예측 (단일 항구 처리 포함)
    for cluster, prob_cluster in enumerate(cluster_probs):
        if cluster not in cluster_model_paths:
            continue

        try:
            model = joblib.load(cluster_model_paths[cluster]["model"])
            le = joblib.load(cluster_model_paths[cluster]["encoder"])
        except FileNotFoundError:
            print(f"⚠️ 군집 {cluster}의 모델 또는 인코더를 불러올 수 없습니다.")
            continue

        # 항구가 1개일 경우 → 직접 확률 계산 없이 100% 확률로 넣기
        port_classes = le.classes_
        if len(port_classes) == 1:
            single_port = port_classes[0]
            joint_prob = prob_cluster * 1.0  # 100%
            final_probs[single_port] = final_probs.get(single_port, 0) + joint_prob
            print(f"📦 군집 {cluster}은 단일 항구 '{single_port}' → 확률 곱 적용: {joint_prob:.2%}")
            continue

        # 항구가 여러 개일 경우 → 모델 예측 수행
        port_probs = model.predict_proba(user_input)[0]
        port_names = le.inverse_transform(np.arange(len(port_probs)))

        for port, prob_port in zip(port_names, port_probs):
            joint_prob = prob_cluster * prob_port
            final_probs[port] = final_probs.get(port, 0) + joint_prob

    # ▶ Top-3 항구 출력
    if not final_probs:
        print("❗ 예측 가능한 항구가 없습니다.")
        return

    sorted_ports = sorted(final_probs.items(), key=lambda x: x[1], reverse=True)[:3]

    print("\n🔮 [Joint 확률 기반] Top-3 항구 예측 결과:")
    for port, prob in sorted_ports:
        print(f"📦 {port}: {prob:.2%}")

In [81]:
predict_next_port_joint_prob()

🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)


LAT (위도):  34.656852
LON (경도):  129.62240
COG (침로):  155.97964
HEADING (타각):  153.11976



🔍 [1차 분류기] 군집별 확률:
 - CLUSTER_0: 0.92%
 - CLUSTER_1: 77.79%
 - CLUSTER_2: 2.09%
 - CLUSTER_3: 1.46%
 - CLUSTER_4: 14.39%
 - CLUSTER_5: 0.61%
 - CLUSTER_6: 0.78%
 - CLUSTER_7: 1.96%

🔮 [Joint 확률 기반] Top-3 항구 예측 결과:
📦 JPHKT: 74.18%
📦 JPHIJ: 2.69%
📦 RUVVO: 1.92%


# 재시도
- 비율을 재조정 하고 싶을 때

In [87]:
import numpy as np
import joblib

# 🔁 군집별 모델 경로 딕셔너리
cluster_model_paths = {
    0: {"model": "models/cluster_0_rf.joblib", "encoder": "models/cluster_0_label_encoder.joblib"},
    1: {"model": "models/cluster_1_rf_xgb_knn_lightgbm.joblib", "encoder": "models/cluster_1_label_encoder.joblib"},
    2: {"model": "models/cluster_2_rf.joblib", "encoder": "models/cluster_2_label_encoder.joblib"},
    7: {"model": "models/cluster_7_rf.joblib", "encoder": "models/cluster_7_label_encoder.joblib"},
}

def predict_next_port_joint_prob(weight_cluster=0.3, weight_port=0.7):
    print("🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)")
    try:
        lat = float(input("LAT (위도): "))
        lon = float(input("LON (경도): "))
        cog = float(input("COG (침로): "))
        heading = float(input("HEADING (타각): "))
    except ValueError:
        print("⚠️ 숫자만 입력해 주세요.")
        return

    user_input = np.array([[lat, lon, cog, heading]])

    # ▶ 1차 분류기 로드 및 CLUSTER_1 확률 예측
    try:
        cluster_model = joblib.load("models/softvoting_cluster1_model.joblib")
    except FileNotFoundError:
        print("❌ 1차 분류기 모델이 없습니다.")
        return

    cluster_probs = cluster_model.predict_proba(user_input)[0]
    print("\n🔍 [1차 분류기] 군집별 확률:")
    for i, p in enumerate(cluster_probs):
        print(f" - CLUSTER_{i}: {p:.2%}")

    final_probs = {}

    # ▶ 각 군집별로 항구 예측 (단일 항구 처리 + 보정 확률 계산)
    for cluster, prob_cluster in enumerate(cluster_probs):
        if cluster not in cluster_model_paths:
            continue

        try:
            model = joblib.load(cluster_model_paths[cluster]["model"])
            le = joblib.load(cluster_model_paths[cluster]["encoder"])
        except FileNotFoundError:
            print(f"⚠️ 군집 {cluster}의 모델 또는 인코더를 불러올 수 없습니다.")
            continue

        port_classes = le.classes_
        if len(port_classes) == 1:
            single_port = port_classes[0]
            joint_prob = (prob_cluster * weight_cluster) + (1.0 * weight_port)
            final_probs[single_port] = final_probs.get(single_port, 0) + joint_prob
            print(f"📦 군집 {cluster}은 단일 항구 '{single_port}' → 보정 확률 적용: {joint_prob:.2%}")
            continue

        # 항구가 여러 개인 경우
        port_probs = model.predict_proba(user_input)[0]
        port_names = le.inverse_transform(np.arange(len(port_probs)))

        for port, prob_port in zip(port_names, port_probs):
            joint_prob = (prob_cluster * weight_cluster) + (prob_port * weight_port)
            final_probs[port] = final_probs.get(port, 0) + joint_prob

    # ▶ Top-3 항구 출력
    if not final_probs:
        print("❗ 예측 가능한 항구가 없습니다.")
        return

    sorted_ports = sorted(final_probs.items(), key=lambda x: x[1], reverse=True)[:3]

    print("\n🔮 [가중 평균 기반] Top-3 항구 예측 결과:")
    for port, prob in sorted_ports:
        print(f"📦 {port}: {prob:.2%}")


In [89]:
predict_next_port_joint_prob()

🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)


LAT (위도):  34.0297233
LON (경도):  128.60626
COG (침로):  195.8
HEADING (타각):  198



🔍 [1차 분류기] 군집별 확률:
 - CLUSTER_0: 26.94%
 - CLUSTER_1: 13.08%
 - CLUSTER_2: 30.17%
 - CLUSTER_3: 1.43%
 - CLUSTER_4: 9.52%
 - CLUSTER_5: 6.68%
 - CLUSTER_6: 1.47%
 - CLUSTER_7: 10.71%

🔮 [가중 평균 기반] Top-3 항구 예측 결과:
📦 RUVVO: 73.21%
📦 CNSHA: 59.45%
📦 KRYOS: 58.87%


# Mapping 한 경우

In [38]:
import numpy as np
import joblib

# 군집별 모델 경로 딕셔너리
cluster_model_paths = {
    1: {"model": "models/port_model_1.joblib", "encoder": "models/encoder_1.joblib"},
    2: {"model": "models/port_model_2.joblib", "encoder": "models/encoder_2.joblib"},
    3: {"model": "models/port_model_3.joblib", "encoder": "models/encoder_3.joblib"},
    4: {"model": "models/port_model_4.joblib", "encoder": "models/encoder_4.joblib"},
    6: {"model": "models/port_model_6.joblib", "encoder": "models/encoder_6.joblib"},
    7: {"model": "models/port_model_7.joblib", "encoder": "models/encoder_7.joblib"}
}

# ▶ 고정된 군집-항구 매핑 정의
fixed_cluster_ports = {
  0: "PHMNL",
  5: "VNHPH"
}


def predict_next_port_joint_prob():
    print("🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)")
    try:
        lat = float(input("LAT (위도): "))
        lon = float(input("LON (경도): "))
        cog = float(input("COG (침로): "))
        heading = float(input("HEADING (타각): "))
    except ValueError:
        print("⚠️ 숫자만 입력해 주세요.")
        return

    user_input = np.array([[lat, lon, cog, heading]])

    # ▶ 1차 분류기 로드 및 CLUSTER_1 확률 예측
    try:
        cluster_model = joblib.load("./models/cluster_model.joblib")
    except FileNotFoundError:
        print("❌ 1차 분류기 모델이 없습니다.")
        return

    cluster_probs = cluster_model.predict_proba(user_input)[0]
    cluster_labels = cluster_model.classes_
    
    # ▶ Top-5 군집만 추출
    top_n = 5
    top_idx = set(np.argsort(cluster_probs)[-top_n:])
    # 고정 군집이 cluster_labels 안에 있는 경우만 포함
    for fixed_cluster in fixed_cluster_ports:
        if fixed_cluster in cluster_labels:
            fixed_idx = np.where(cluster_labels == fixed_cluster)[0][0]
            top_idx.add(fixed_idx)

    top_idx = list(top_idx)

    print("\n🔍 [1차 분류기] Top 군집 + 고정 포함:")
    for i in top_idx:
        print(f" - CLUSTER_{cluster_labels[i]}: {cluster_probs[i]:.2%}")

    final_probs = {}

    for i in top_idx:
        prob_cluster = cluster_probs[i]
        cluster_id = cluster_labels[i]

        # ▶ 고정 항구 매핑 처리
        if cluster_id in fixed_cluster_ports:
            fixed_port = fixed_cluster_ports[cluster_id]
            joint_prob = prob_cluster * 1.0
            final_probs[fixed_port] = final_probs.get(fixed_port, 0) + joint_prob
            print(f"📦 군집 {cluster_id}은 고정 항구 '{fixed_port}' → 확률 곱 적용: {joint_prob:.2%}")
            continue

        # ▶ 2차 모델 불러오기
        if cluster_id not in cluster_model_paths:
            print(f"⚠️ 군집 {cluster_id}의 모델 경로가 정의되지 않았습니다.")
            continue

        try:
            model = joblib.load(cluster_model_paths[cluster_id]["model"])
            le = joblib.load(cluster_model_paths[cluster_id]["encoder"])
        except FileNotFoundError:
            print(f"⚠️ 군집 {cluster_id}의 모델 또는 인코더를 불러올 수 없습니다.")
            continue

        # ▶ 항구 예측 및 Joint 확률 계산
        port_probs = model.predict_proba(user_input)[0]
        try:
            port_names = le.inverse_transform(np.arange(len(port_probs)))
        except:
            port_names = model.classes_

        for port, prob_port in zip(port_names, port_probs):
            joint_prob = prob_cluster * prob_port
            final_probs[port] = final_probs.get(port, 0) + joint_prob

    # ▶ Top-3 항구 출력
    if not final_probs:
        print("❗ 예측 가능한 항구가 없습니다.")
        return

    sorted_ports = sorted(final_probs.items(), key=lambda x: x[1], reverse=True)[:3]

    print("\n🔮 [Joint 확률 기반] Top-3 항구 예측 결과:")
    for port, prob in sorted_ports:
        print(f"📦 {port}: {prob:.2%}")

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [72]:
predict_next_port_joint_prob()

🚢 AIS 데이터 입력해주세요 (5시간 시점 기준)


LAT (위도):  34.15177
LON (경도):  128.45687667
COG (침로):  229.9
HEADING (타각):  232



🔍 [1차 분류기] Top 군집 + 고정 포함:
 - CLUSTER_0: 0.19%
 - CLUSTER_1: 1.25%
 - CLUSTER_2: 60.86%
 - CLUSTER_4: 2.62%
 - CLUSTER_5: 6.86%
 - CLUSTER_6: 27.85%
📦 군집 0은 고정 항구 'PHMNL' → 확률 곱 적용: 0.19%
📦 군집 5은 고정 항구 'VNHPH' → 확률 곱 적용: 6.86%

🔮 [Joint 확률 기반] Top-3 항구 예측 결과:
📦 CNTAC: 48.25%
📦 CNQDG: 15.61%
📦 CNLYG: 9.12%
